In [1]:
#Importing libraries
import re
import numpy as np
import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings 
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('cars_cleaned.csv')

In [3]:
df.shape

(32316, 48)

In [4]:
df.head()

,MSRP,EPA Fuel Economy Est - City (MPG),Engine,Drivetrain,Passenger Capacity,Passenger Doors,Base Curb Weight (lbs),Passenger Volume (ft³),Wheelbase (in),"Track Width, Front (in)","Height, Overall (in)","Fuel Tank Capacity, Approx (gal)",SAE Net Torque @ RPM,Fuel System,SAE Net Horsepower @ RPM,Displacement,Trans Description Cont.,Trans Type,Suspension Type - Front,Suspension Type - Rear,Air Bag-Frontal-Driver,Air Bag-Frontal-Passenger,Air Bag-Passenger Switch (On/Off),Air Bag-Side Body-Front,Air Bag-Side Body-Rear,Air Bag-Side Head-Front,Air Bag-Side Head-Rear,Brakes-ABS,Child Safety Rear Door Locks,Daytime Running Lights,Traction Control,Night Vision,Rollover Protection Bars,Fog Lamps,Parking Aid,Tire Pressure Monitor,Back-Up Camera,Stability Control,Basic Miles/km,Basic Years,Corrosion Miles/km,Corrosion Years,Drivetrain Miles/km,Drivetrain Years,Turning Diameter - Curb to Curb (ft),Manufacturer,Model year,Category
0,40600.0,22.0,l4,FWD,5,4,3790.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
1,45500.0,22.0,l4,FWD,5,4,3829.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
2,43600.0,22.0,l4,FWD,5,4,3821.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
3,37400.0,22.0,l4,FWD,5,4,3783.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,0,0,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
4,42600.0,21.0,l4,AWD,5,4,4026.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV


In [5]:
def missing_values(dataframe):
    null_counts = dataframe.isnull().sum()
    mean_missing = dataframe.isnull().mean()*100
    missing_val = pd.DataFrame({'Count' : null_counts[null_counts > 0] , 'Percentage Missing(%)' : mean_missing[mean_missing > 0] })
    missing_val.sort_values(by = 'Count' , inplace=True)
    missing_val.reset_index(inplace=True)
    missing_val.columns = ['Features' , 'Count' , 'Percentage Missing(%)' ]
    return missing_val
missing = missing_values(df)
missing

,Features,Count,Percentage Missing(%)
0,MSRP,54,0.167100
1,Drivetrain,1716,5.310063
2,Category,1716,5.310063
3,Engine,1975,6.111524
4,Trans Type,1981,6.130090
5,SAE Net Horsepower @ RPM,2014,6.232207
6,Wheelbase (in),2015,6.235301
7,Trans Description Cont.,2037,6.303379
8,SAE Net Torque @ RPM,2067,6.396212
9,Basic Years,2117,6.550935


##### Category columns that needs to be imputed

In [6]:
cat_col_imp = ['Drivetrain' , 'Category' , 'Engine' , 'Trans Type']

##### Dropping Rows

The rows in which price and category were missing are removed.

In [7]:
df.dropna(axis=0 , how = 'any' , subset=['MSRP','Drivetrain'] , inplace=True)

In [8]:
df[['MSRP' , 'Drivetrain']].isnull().sum()

MSRP          0
Drivetrain    0
dtype: int64

##### Engine
Imputing the missing engine values by taking the most occurrent value after grouping them together by their manufacturer and category

In [9]:
#Imuting missing values in egnine by grouping them by manufacturer and category wise
df['Engine'] = df['Engine'].fillna(df.groupby(['Manufacturer', 'Category'])['Engine'].transform(lambda x: x.value_counts().idxmax()))

##### Transmission type (No of gears)

In [10]:
#Trans type
df['Trans Type'].value_counts(dropna=False)

6      11420
5       7459
4       4663
8       3000
<3      1663
7       1374
9        487
NaN      265
10       193
CVT       22
Name: Trans Type, dtype: int64

In [11]:
##Imuting missing values in transmission type by grouping them by manufacturer and engine wise
df['Trans Type'] = df['Trans Type'].fillna(df.groupby(['Manufacturer','Category','Engine'])['Trans Type'].transform(lambda x: x.value_counts().idxmax()))

##### Type of transmission used

In [12]:
df['Trans Description Cont.'].value_counts(dropna= False)

Automatic    19063
Manual        9566
CVT           1596
NaN            321
Name: Trans Description Cont., dtype: int64

In [13]:
df['Trans Description Cont.'] = df['Trans Description Cont.'].fillna(df.groupby(['Manufacturer','Category','Engine'])['Trans Description Cont.'].transform(lambda x: x.value_counts().idxmax()))

##### Fuel System

In [14]:
df['Fuel System'].value_counts(dropna=False)

SFI            12561
DI             10600
Electric FI     6275
NaN             1110
Name: Fuel System, dtype: int64

In [15]:
df['Fuel System'] = df['Fuel System'].fillna('Unknown')

In [16]:
#Remaining columns tht has to be imputed
num_col_imp = ['EPA Fuel Economy Est - City (MPG)', 'Base Curb Weight (lbs)',
               'Passenger Volume (ft³)', 'Wheelbase (in)', 'Track Width, Front (in)',
               'Height, Overall (in)', 'Fuel Tank Capacity, Approx (gal)',
               'SAE Net Torque @ RPM', 'Fuel System', 'SAE Net Horsepower @ RPM',
               'Displacement', 'Basic Miles/km',
               'Basic Years', 'Corrosion Miles/km', 'Corrosion Years',
               'Drivetrain Miles/km', 'Drivetrain Years',
               'Turning Diameter - Curb to Curb (ft)']

In [17]:
df.head()

,MSRP,EPA Fuel Economy Est - City (MPG),Engine,Drivetrain,Passenger Capacity,Passenger Doors,Base Curb Weight (lbs),Passenger Volume (ft³),Wheelbase (in),"Track Width, Front (in)","Height, Overall (in)","Fuel Tank Capacity, Approx (gal)",SAE Net Torque @ RPM,Fuel System,SAE Net Horsepower @ RPM,Displacement,Trans Description Cont.,Trans Type,Suspension Type - Front,Suspension Type - Rear,Air Bag-Frontal-Driver,Air Bag-Frontal-Passenger,Air Bag-Passenger Switch (On/Off),Air Bag-Side Body-Front,Air Bag-Side Body-Rear,Air Bag-Side Head-Front,Air Bag-Side Head-Rear,Brakes-ABS,Child Safety Rear Door Locks,Daytime Running Lights,Traction Control,Night Vision,Rollover Protection Bars,Fog Lamps,Parking Aid,Tire Pressure Monitor,Back-Up Camera,Stability Control,Basic Miles/km,Basic Years,Corrosion Miles/km,Corrosion Years,Drivetrain Miles/km,Drivetrain Years,Turning Diameter - Curb to Curb (ft),Manufacturer,Model year,Category
0,40600.0,22.0,l4,FWD,5,4,3790.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
1,45500.0,22.0,l4,FWD,5,4,3829.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
2,43600.0,22.0,l4,FWD,5,4,3821.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
3,37400.0,22.0,l4,FWD,5,4,3783.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,0,0,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV
4,42600.0,21.0,l4,AWD,5,4,4026.0,104.0,108.3,64.2,65.7,17.1,280.0,DI,272.0,2.0,Automatic,10,MacPherson Strut,Link type,1,1,0,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,50000.0,4.0,150000.0,5.0,70000.0,NaN,39.0,Acura,2019,SUV


In [18]:
df.to_csv('Cars_cleaned_wo_OHE.csv',index=False)

##### One hot Encoding all categorical columns

In [19]:
#One hot encoding
#If a category has more than 300 values then a new column is created
df1 = df.copy()

cat_col = ['Engine' , 'Drivetrain' , 'Trans Description Cont.' , 'Fuel System','Suspension Type - Front',
           'Trans Type','Suspension Type - Rear','Manufacturer','Category']
print('The Encoding is applied for: ')
for col in cat_col:
    freqs=df1[col].value_counts()
    k=freqs.index[freqs>300]
    for cat in k:
        name=col+'_'+cat
        df1[name]=(df1[col]==cat).astype(int)
    del df1[col]
    print(col)

The Encoding is applied for: 
Engine
Drivetrain
Trans Description Cont.
Fuel System
Suspension Type - Front
Trans Type
Suspension Type - Rear
Manufacturer
Category


In [20]:
# df2 = df1.copy()
# #MICE IMPUTATION
# from fancyimpute import IterativeImputer
# MICE_imputer = IterativeImputer()

# df2.iloc[:,:] = MICE_imputer.fit_transform(df1)

# df2.to_csv('Car_MICE_imp.csv',index=False)

In [21]:
# df3 = df1.copy()
# from fancyimpute import KNN

# # Initialize KNN
# knn_imputer = KNN()

# # Impute using fit_tranform on diabetes_knn_imputed
# df3.iloc[:, :] = knn_imputer.fit_transform(df1)

In [22]:
# df2.to_csv('Car_MICE_imp.csv',index=False)

In [25]:
mice = pd.read_csv('Car_MICE_imp.csv')
mice.head()

,MSRP,EPA Fuel Economy Est - City (MPG),Passenger Capacity,Passenger Doors,Base Curb Weight (lbs),Passenger Volume (ft³),Wheelbase (in),"Track Width, Front (in)","Height, Overall (in)","Fuel Tank Capacity, Approx (gal)",SAE Net Torque @ RPM,SAE Net Horsepower @ RPM,Displacement,Air Bag-Frontal-Driver,Air Bag-Frontal-Passenger,Air Bag-Passenger Switch (On/Off),Air Bag-Side Body-Front,Air Bag-Side Body-Rear,Air Bag-Side Head-Front,Air Bag-Side Head-Rear,Brakes-ABS,Child Safety Rear Door Locks,Daytime Running Lights,Traction Control,Night Vision,Rollover Protection Bars,Fog Lamps,Parking Aid,Tire Pressure Monitor,Back-Up Camera,Stability Control,Basic Miles/km,Basic Years,Corrosion Miles/km,Corrosion Years,Drivetrain Miles/km,Drivetrain Years,Turning Diameter - Curb to Curb (ft),Model year,Engine_l4,Engine_V8,Engine_V6,Engine_Flat,Engine_I6,Engine_l5,Drivetrain_RWD,Drivetrain_FWD,Drivetrain_4WD,Drivetrain_AWD,Trans Description Cont._Automatic,Trans Description Cont._Manual,Trans Description Cont._CVT,Fuel System_SFI,Fuel System_DI,Fuel System_Electric FI,Fuel System_Unknown,Suspension Type - Front_MacPherson Strut,Suspension Type - Front_Double Wishbone,Suspension Type - Front_Independent,Suspension Type - Front_Coil Spring,Suspension Type - Front_Link type,Suspension Type - Front_Torsion Bar,Suspension Type - Front_Others,Trans Type_6,Trans Type_5,Trans Type_4,Trans Type_8,Trans Type_<3,Trans Type_7,Trans Type_9,Suspension Type - Rear_Link type,Suspension Type - Rear_Leaf type,Suspension Type - Rear_Independent,Suspension Type - Rear_Double Wishbone,Suspension Type - Rear_Torsion Bar,Suspension Type - Rear_Others,Suspension Type - Rear_Coil Spring,Suspension Type - Rear_MacPherson Strut,Suspension Type - Rear_Axle,Manufacturer_Ford,Manufacturer_Chevrolet,Manufacturer_Toyota,Manufacturer_Nissan,Manufacturer_GMC,Manufacturer_Honda,Manufacturer_Volkswagen,Manufacturer_Subaru,Manufacturer_BMW,Manufacturer_Audi,Manufacturer_Ram,Manufacturer_Hyundai,Manufacturer_Mercedes-Benz,Manufacturer_Mazda,Manufacturer_Kia,Manufacturer_Jeep,Manufacturer_Cadillac,Manufacturer_Porsche,Manufacturer_Volvo,Manufacturer_Dodge,Manufacturer_Jaguar,Manufacturer_Buick,Manufacturer_Acura,Category_Car,Category_Pickup,Category_SUV,Category_Van
0,40600.0,22.0,5.0,4.0,3790.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.642347,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,45500.0,22.0,5.0,4.0,3829.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.649791,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,43600.0,22.0,5.0,4.0,3821.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.654208,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,37400.0,22.0,5.0,4.0,3783.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.752583,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0

In [24]:
knn = pd.read_csv('Car_KNN_imp.csv')
knn.head()

,MSRP,EPA Fuel Economy Est - City (MPG),Passenger Capacity,Passenger Doors,Base Curb Weight (lbs),Passenger Volume (ft³),Wheelbase (in),"Track Width, Front (in)","Height, Overall (in)","Fuel Tank Capacity, Approx (gal)",SAE Net Torque @ RPM,SAE Net Horsepower @ RPM,Displacement,Air Bag-Frontal-Driver,Air Bag-Frontal-Passenger,Air Bag-Passenger Switch (On/Off),Air Bag-Side Body-Front,Air Bag-Side Body-Rear,Air Bag-Side Head-Front,Air Bag-Side Head-Rear,Brakes-ABS,Child Safety Rear Door Locks,Daytime Running Lights,Traction Control,Night Vision,Rollover Protection Bars,Fog Lamps,Parking Aid,Tire Pressure Monitor,Back-Up Camera,Stability Control,Basic Miles/km,Basic Years,Corrosion Miles/km,Corrosion Years,Drivetrain Miles/km,Drivetrain Years,Turning Diameter - Curb to Curb (ft),Model year,Engine_l4,Engine_V8,Engine_V6,Engine_Flat,Engine_I6,Engine_l5,Drivetrain_RWD,Drivetrain_FWD,Drivetrain_4WD,Drivetrain_AWD,Trans Description Cont._Automatic,Trans Description Cont._Manual,Trans Description Cont._CVT,Fuel System_SFI,Fuel System_DI,Fuel System_Electric FI,Fuel System_Unknown,Suspension Type - Front_MacPherson Strut,Suspension Type - Front_Double Wishbone,Suspension Type - Front_Independent,Suspension Type - Front_Coil Spring,Suspension Type - Front_Link type,Suspension Type - Front_Torsion Bar,Suspension Type - Front_Others,Trans Type_6,Trans Type_5,Trans Type_4,Trans Type_8,Trans Type_<3,Trans Type_7,Trans Type_9,Suspension Type - Rear_Link type,Suspension Type - Rear_Leaf type,Suspension Type - Rear_Independent,Suspension Type - Rear_Double Wishbone,Suspension Type - Rear_Torsion Bar,Suspension Type - Rear_Others,Suspension Type - Rear_Coil Spring,Suspension Type - Rear_MacPherson Strut,Suspension Type - Rear_Axle,Manufacturer_Ford,Manufacturer_Chevrolet,Manufacturer_Toyota,Manufacturer_Nissan,Manufacturer_GMC,Manufacturer_Honda,Manufacturer_Volkswagen,Manufacturer_Subaru,Manufacturer_BMW,Manufacturer_Audi,Manufacturer_Ram,Manufacturer_Hyundai,Manufacturer_Mercedes-Benz,Manufacturer_Mazda,Manufacturer_Kia,Manufacturer_Jeep,Manufacturer_Cadillac,Manufacturer_Porsche,Manufacturer_Volvo,Manufacturer_Dodge,Manufacturer_Jaguar,Manufacturer_Buick,Manufacturer_Acura,Category_Car,Category_Pickup,Category_SUV,Category_Van
0,40600.0,22.0,5.0,4.0,3790.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.0,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,45500.0,22.0,5.0,4.0,3829.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.0,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,43600.0,22.0,5.0,4.0,3821.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.0,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,37400.0,22.0,5.0,4.0,3783.0,104.0,108.3,64.2,65.7,17.1,280.0,272.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,50000.0,4.0,150000.0,5.0,70000.0,6.0,39.0,2019.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0